# Water Quality and Potability Classification
  
#### [Dataset URL](https://www.kaggle.com/datasets/uom190346a/water-quality-and-potability)
  
## Opis zbioru danych

Ten zbiór danych zawiera pomiary jakości wody oraz oceny dotyczące jej zdatności do spożycia przez ludzi, czyli potencjał pitności. Głównym celem tego zbioru danych jest dostarczenie wglądu w parametry jakości wody i pomoc w określeniu, czy woda jest zdatna do spożycia. Każdy wiersz w zbiorze danych reprezentuje próbkę wody z określonymi cechami, a kolumna "Potability" wskazuje, czy woda jest odpowiednia do spożycia. Głównym celem tego zbioru danych jest ocena i przewidywanie potencjału potabilności wody na podstawie cech jakości wody. Może być używany do oceny bezpieczeństwa i odpowiedniości źródeł wody do spożycia przez ludzi, podejmowania świadomych decyzji dotyczących uzdatniania wody oraz zapewnienia zgodności z normami jakości wody.

## Opis cech

- pH: Poziom pH wody.
- Hardness: Twardość wody, miara zawartości minerałów.
- Solids: Całkowita zawartość substancji rozpuszczonych w wodzie.
- Chloramines: Stężenie chloramin w wodzie.
- Sulfate: Stężenie siarczanów w wodzie.
- Conductivity: Przewodność elektryczna wody.
- Organic_carbon: Zawartość węgla organicznego w wodzie.
- Trihalomethanes: Stężenie trihalometanów w wodzie.
- Turbidity: Poziom mętności, miara klarowności wody.
- Potability: Zmienna celu; wskazuje zdatność do spożycia wody, przyjmując wartości 1 (zdatna do spożycia - "potable") i 0 (niezdatna do spożycia - "not potable).

## Parametry zbioru danych

- Liczba rekordów: 3276
- Liczba cech: 9
- Dane brakujące: Tak (kolumny: pH, Sulfate and Trihalomethanes)
- Dane odstające: Tak (ok. 1.22% całego zbioru danych) 
- Typ problemu: Klasyfikacja (Potability - No (0), Yes (1))

## Rozkład klas

| Klasa | Liczba rekordów | Rozkład procentowy |
|-------|-----------------|--------------------|
| 0     | 1998            | 60.99%             |
| 1     | 1278            | 39.01%             |

## Importowanie bibliotek

In [2]:
import pickle
import random
import time

import numpy as np
import optuna
from deap import base, creator, tools, algorithms
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

## Załadowanie zmiennych

Zmienne zostają załadowane z pliku wygenerowanego z notebooka DataAnalysis.ipynb przy pomocy biblioteki pickle, służącej do serializacji oraz deserializacji danych. Wybrano dane, które osiągnęły najlepsze wyniki podczas 2 etapu - tworzenia i trenowania modeli.

In [3]:
with open('data_dump/normalizedStdInterpolateVars.pkl', 'rb') as f:
    normalized_std_interpolate = pickle.load(f)
    scaler_std_interpolate = pickle.load(f)

## Funkcje do podziału zbioru danych

Utworzono dwie funkcje do tworzenia podziału danych. Split_df_train_test odpowiada za podział danych na dwa zbiory: testowy i treningowy. Dane są dzielone z równomiernym podziałem klas, aby zapobiec sytuacji, w której podczas podziału danych, przydzielono do zbioru treningowego tylko jedną klasę danych. 

In [4]:
def split_df_train_test(data, test_size, seed):
    np.random.seed(seed)

    unique_labels = data['Potability'].unique()
    label_counts = data['Potability'].value_counts()

    test_indices = []

    for label in unique_labels:
        num_label_samples = label_counts[label]
        num_test_samples = int(test_size * num_label_samples)
        label_indices = data.index[data['Potability'] == label].tolist()
        label_test_indices = np.random.choice(label_indices, size=num_test_samples, replace=False)
        test_indices.extend(label_test_indices)

    train_indices = np.setdiff1d(data.index, test_indices)

    train_set = data.loc[train_indices]
    test_set = data.loc[test_indices]

    return train_set, test_set

### Funkcja do obliczania metryki F1-score

In [5]:
def calculate_f1_score(y_true, y_pred):
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    true_negatives = np.sum((y_true == 0) & (y_pred == 0))
    false_positives = np.sum((y_true == 0) & (y_pred == 1))
    false_negatives = np.sum((y_true == 1) & (y_pred == 0))

    precision_positives = true_positives / (true_positives + false_positives)
    recall_positives = true_positives / (true_positives + false_negatives)
    f1_score_positives = 2 * (precision_positives * recall_positives) / (precision_positives + recall_positives)

    precision_negatives = true_negatives / (true_negatives + false_negatives)
    recall_negatives = true_negatives / (true_negatives + false_positives)
    f1_score_negatives = 2 * (precision_negatives * recall_negatives) / (precision_negatives + recall_negatives)

    f1_score = (f1_score_positives + f1_score_negatives) / 2
    return f1_score

### Split danych

Podzielenie danych przy pomocy funkcji "split_df_train_test"

In [6]:
train_std_interpolate, test_std_interpolate = split_df_train_test(normalized_std_interpolate, 0.2, 123)

### Podział klas po splicie - test/train

In [7]:
train_class_counts = train_std_interpolate['Potability'].value_counts(normalize=True) * 100
test_class_counts = test_std_interpolate['Potability'].value_counts(normalize=True) * 100

print("Train set:")
print(train_class_counts)
print("\nTest set:")
print(test_class_counts)

Train set:
Potability
0    60.983982
1    39.016018
Name: proportion, dtype: float64

Test set:
Potability
0    61.009174
1    38.990826
Name: proportion, dtype: float64


## SVM - Scikit-Learn

In [8]:
class SVMClassifierWrapper:

    def __init__(self, train_set, test_set):
        self.train_set = train_set.iloc[:, :-1]
        self.test_set = test_set.iloc[:, :-1]
        self.train_label = train_set.iloc[:, -1]
        self.test_label = test_set.iloc[:, -1]
        self.model = None
        self.history = None
        self.test_pred = None
        self.hof = None
        self.best_optuna_trial = None
        self._create_model()

    def _create_model(self):
        self.model = SVC(random_state=42)

    def evalSVMModel(self, individual, optimize_f1_score):
        C, degree, coef0 = individual
        degree = int(degree)
        C = max(C, 0.1)
        coef0 = max(coef0, 0)
        model = SVC(C=C, degree=degree, coef0=coef0, random_state=42)
        model.fit(self.train_set, self.train_label)
        predictions = model.predict(self.test_set)

        if optimize_f1_score:
            score = calculate_f1_score(self.test_label, predictions)
        else:
            score = accuracy_score(self.test_label, predictions)

        return score,

    def train_model_with_ga(self, optimize_f1_score=False):
        if not hasattr(creator, 'FitnessMax'):
            creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        if not hasattr(creator, 'Individual'):
            creator.create("Individual", list, fitness=creator.FitnessMax)

        toolbox = base.Toolbox()

        toolbox.register("attr_C", random.uniform, 0.1, 30)
        toolbox.register("attr_degree", random.randint, 1, 10)
        toolbox.register("attr_coef0", random.uniform, 0, 20)

        toolbox.register("individual", tools.initCycle, creator.Individual,
                         (toolbox.attr_C, toolbox.attr_degree, toolbox.attr_coef0), n=1)

        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        toolbox.register("evaluate", self.evalSVMModel, optimize_f1_score=optimize_f1_score)
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
        toolbox.register("select", tools.selTournament, tournsize=5)

        pop = toolbox.population(n=100)
        self.hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("min", np.min)
        stats.register("max", np.max)

        #toolbox.register("map", futures.map) <- scoop library
        pop, logbook = algorithms.eaMuPlusLambda(pop, toolbox, mu=50, lambda_=50, cxpb=0.5, mutpb=0.2, ngen=5,
                                                 stats=stats, halloffame=self.hof, verbose=True)

        best_individual = tools.selBest(pop, 1)[0]

        self.model = SVC(C=best_individual[0], degree=int(best_individual[1]), coef0=best_individual[2], kernel="poly", random_state=42)
        self.model.fit(self.train_set, self.train_label)

    def optuna_objective(self, trial, optimize_f1_score):
        C = trial.suggest_float("C", 0.1, 30)
        degree = trial.suggest_int("degree", 1, 10)
        coef0 = trial.suggest_float("coef0", 0, 20)

        model = SVC(C=C, degree=degree, coef0=coef0, kernel="poly", random_state=42)
        model.fit(self.train_set, self.train_label)
        predictions = model.predict(self.test_set)

        if optimize_f1_score:
            score = calculate_f1_score(self.test_label, predictions)
        else:
            score = accuracy_score(self.test_label, predictions)

        return score

    def train_model_with_optuna(self, optimize_f1_score=False, n_trials=5):
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda trial: self.optuna_objective(trial, optimize_f1_score), n_jobs=-1, n_trials=n_trials)

        self.best_optuna_trial = study.best_trial

        self.model = SVC(C=self.best_optuna_trial.params["C"], degree=self.best_optuna_trial.params["degree"],
                         coef0=self.best_optuna_trial.params["coef0"], kernel="poly", random_state=42)
        self.model.fit(self.train_set, self.train_label)

    def get_deap_best_individual(self):
        return self.hof[0]
    
    def get_optuna_best_result(self):
        return self.best_optuna_trial

### Trenowanie modelu SVM - optymalizacja hiperparametrów za pomocą algorytmu genetycznego

In [9]:
def print_best_individual(model):
    best_individual = model.get_deap_best_individual()
    print("Best hiperparameters:")
    print(f"C: {best_individual[0]}")
    print(f"Degree: {best_individual[1]}")
    print(f"Coef0: {best_individual[2]}")

print("DEAP - Accuracy: \n")
start_time = time.time()
SVM_model_accuracy_optimizing_DEAP = SVMClassifierWrapper(train_std_interpolate, test_std_interpolate)
SVM_model_accuracy_optimizing_DEAP.train_model_with_ga(optimize_f1_score=False)
print_best_individual(SVM_model_accuracy_optimizing_DEAP)
end_time = time.time()
print(f"DEAP - Execution time for accuracy: {format(end_time - start_time, '.2f')} seconds")

print("-" * 50)

# print("DEAP - F1 Score: \n")
# start_time = time.time()
# SVM_model_f1score_optimizing_DEAP = SVMClassifierWrapper(train_std_interpolate, test_std_interpolate)
# SVM_model_f1score_optimizing_DEAP.train_model_with_ga(optimize_f1_score=True)
# print_best_individual(SVM_model_f1score_optimizing_DEAP)
# end_time = time.time()
# print(f"DEAP - Execution time for F1 Score: {format(end_time - start_time, '.2f')} seconds")

DEAP - Accuracy: 

gen	nevals	avg     	min     	max     
0  	100   	0.675795	0.646789	0.695719
1  	31    	0.686667	0.678899	0.695719
2  	31    	0.692997	0.680428	0.695719
3  	40    	0.695657	0.692661	0.695719
4  	38    	0.695719	0.695719	0.695719
5  	32    	0.695719	0.695719	0.695719
Best hiperparameters:
C: 2.2842021177940457
Degree: 10
Coef0: 15.490211907592483
DEAP - Execution time for accuracy: 64.82 seconds
--------------------------------------------------


### Trenowanie modelu SVM - optymalizacja hiperparametrów za pomocą biblioteki Optuna

In [ ]:
def print_best_optuna_result(model):
    best_trial = model.get_optuna_best_result()
    print("Best hiperparameters:")
    for key, value in best_trial.params.items():
        print(f"  {key}: {value}")
        
print("Optuna - Accuracy: \n")
start_time = time.time()
SVM_model_accuracy_optimizing_optuna = SVMClassifierWrapper(train_std_interpolate, test_std_interpolate)
SVM_model_accuracy_optimizing_optuna.train_model_with_optuna(optimize_f1_score=False)
print_best_optuna_result(SVM_model_accuracy_optimizing_optuna)
end_time = time.time()
print(f"Optuna - Execution time for Accuracy: {format(end_time - start_time, '.2f')} seconds")

print("-" * 50)

# print("Optuna - F1 Score: \n")
# start_time = time.time()
# SVM_model_f1score_optimizing_optuna = SVMClassifierWrapper(train_std_interpolate, test_std_interpolate)
# SVM_model_f1score_optimizing_optuna.train_model_with_optuna(optimize_f1_score=True)
# print_best_optuna_result(SVM_model_f1score_optimizing_optuna)
# end_time = time.time()
# print(f"Optuna - Execution time for F1 Score: {format(end_time - start_time, '.2f')} seconds")

[I 2024-06-02 21:31:52,558] A new study created in memory with name: no-name-f1626d0f-f11d-45d6-bd7c-ada4c731d434


Optuna - Accuracy: 



[I 2024-06-02 21:31:53,872] Trial 3 finished with value: 0.6743119266055045 and parameters: {'C': 13.881195819278206, 'degree': 4, 'coef0': 0.5679374802270609}. Best is trial 3 with value: 0.6743119266055045.
[I 2024-06-02 21:33:02,570] Trial 4 finished with value: 0.6039755351681957 and parameters: {'C': 25.37964772395835, 'degree': 10, 'coef0': 6.806003366524185}. Best is trial 3 with value: 0.6743119266055045.
